In [25]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

data_dict = {
    "Amphetamine": ["Amy", "bennies", "benz", "dexies", "diet pills", "get ups", "pep pills", "wake-ups", "amphetamine"],
    "Cocaine": ["Blow", "coke", "crack", "nose candy", "cocaine", '❄️', '🌨️', '⛄', '💎', '🎱', '🔑', '😛', '🐡'],
    "DMT": ["Dimitri", "dmt"],
    "General": ["Drugs", "drug"],
    "GHB": ["Georgia home boy", "grievous bodily harm", "ghb", "liquid ecstasy", "liquid e", "liquid x"],
    "Heroin": ["Black tar", "brown sugar", "china white", "heroin", "mexican brown", "skag", "white horse", '🤎', '🐉'],
    "Hydrocodone": ["357s", "dro", "fluff", "norco", "vics", "vikes", "watsons", "hydrocodone"],
    "Ketamine": ["Cat valium", "special k", "vitamin k", "ketamine"],
    "Klonopin": ["k-pin", "super valium", "klonopin"],
    "LSD": ["Acid", "blotter acid", "blotter", "electric kool aid", "lucy in the sky with diamonds", "microdot", "tabs", "lsd"],
    "Marijuana": ["420", "blunt", "bud", "dagga", "dope", "ganja", "grass", "green", "hashish", "hash", "hemp", "herb", "mary jane", "pot", "weed", "marijuana", 'smoke', '🍃', '🍁', '🥦', '💨', '🛒', '🍯', '🕯️', 'carts', 'wax', 'dabs', 'extract'],
    "MDMA": ["e", "ecstasy", "happy pill", "love drug", "molly", "vitamin e", "xtc", "mdma"],
    "Mescaline": ["Blue caps", "media luna", "mescal", "mezcakuba", "topi", "mescaline"],
    "Methamphetamine": ["Crank", "crystal", "meth", "shards", "speed", "tweak", "uppers", "methamphetamine", 'meth', '🔮', '💙', '💎', '🧪'],
    "Mushrooms": ["Boomers", "baps", "mushies", "shrooms", "tweezes", "mushrooms", '🍄'],
    "Nitrous Oxide": ["Buzz bomb", "laughing gas", "nitrous", "nox", "whippets", "nitrous oxide"],
    "Opioid": ["Abstral", "acetaminophen", "actiq", "china girl", "codeine", "dance fever", "dilaudid", "duragesic", "exalgo", "fentanyl", "hydromorphone", "lorcet", "lortab", "methadone", "morphine", "murder 8", "onsolis", "oxy", "oxycodone", "oxyContin", "oxymorphone", "percocet", "vicodin", "opioid", 'perc', '💊' , '🅿️', '🔵' , '🍌'],
    "PCP": ["Angel dust", "love boat", "peace pill", "superweed", "pcp"],
    "Peyote": ["Black button", "green button", "hikuli", "hyatari", "peyote"],
    "Ritalin": ["Ritalin"],
    "Steroids": ["Gym candy", "pumpers", "roids", "steroids", 'tren'],
    "Synthetic Cathinones": ["Bath salts", "bloom", "cloud 9", "cloud nine", "cosmic blast", "flakka", "ivory wave", "lunar wave", "vanilla sky", "white lightning", "synthetic cathinones"],
    "Xanax": ["Benzos", "xanies", "z bars", "zanbars", "xanax", 'xans', '💊', '🍫', '🚌']
}
# Define a custom dataset class
class DrugDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Define your sentences and labels
positive_texts = ["After a long day at work, I like to relax with some weed and watch a movie.",
"He was known for his excessive use of coke, often staying up all night partying.",
"They took ecstasy at the rave and danced until sunrise.",
"The addict struggled to break free from the grip of heroin addiction.",
"She had a profound spiritual experience while tripping on acid in the forest.",
"The dangers of meth were evident in the emaciated figures of its chronic users.",
"She struggled with anxiety and relied on xanax to calm her nerves.",
"He underwent a ketamine infusion therapy session to treat his depression.",
"They went camping and consumed magic mushrooms, experiencing vivid hallucinations.",
"The use of angel dust can lead to unpredictable and dangerous behavior.", "They indulged in a line of blow before hitting the nightclub.",
"The concert was an MDMA-fueled celebration of music and love.",
"She experimented with LSD during her college years, seeking new perspectives.",
"He spiraled into addiction after trying meth for the first time.",
"The party was stocked with a variety of drugs, from weed to cocaine.",
"She relied on ketamine to numb the pain of her past trauma.",
"He struggled with withdrawal symptoms after quitting opioids cold turkey.",
"The dangers of synthetic cathinones became apparent after a friend's overdose.",
"She sought refuge in mushrooms, hoping to find inner peace.",
"They sold PCP on the streets to fund their addiction.","Wanna smoke later?",
"Stressed, got xanax?",
"Took acid, it's wild!",
"Where's the coke tonight?",
"Trying MDMA, thoughts?",
"Taking shrooms for mood.",
"Got ketamine? Need numb.",
"Selling some steroids, low on cash.",
"Need sleeping pills, can't sleep.",
"Heroin's risky, be careful.",
'Lets go smoke some weed',
'Can I get some coke',
'Lets go smoke',
'lets smoke some pcp']
for drug_terms in data_dict.values():
    positive_texts.extend(drug_terms)
negative_texts = ["The sun is shining brightly outside.",
    "I enjoy reading books in my spare time.",
    "Apples are a delicious and healthy snack.",
    "Exercise is important for maintaining good health.",
    "I like to spend time with my family on weekends.",
    "Learning new skills can be both fun and rewarding.",
    "Coffee is my go-to drink in the morning.",
    "The internet provides a wealth of information on various topics.",
    "Traveling allows us to experience different cultures and cuisines.",
    "I enjoy listening to music while I work.", "The cat is sleeping on the windowsill.",
    "I enjoy gardening in my backyard.",
    "Reading a good book is a great way to relax.",
    "I like to cook new recipes on weekends.",
    "The movie I watched last night was very entertaining.",
    "Playing sports is a fun way to stay active.",
    "I'm planning a hiking trip for next month.",
    "Spending time with friends brings me joy.",
    "I'm learning to play the guitar in my spare time.",
    "Volunteering at the local shelter is rewarding.", "The flowers in the garden are blooming beautifully.",
    "I'm looking forward to visiting my grandparents this weekend.",
    "Studying hard is important for academic success.",
    "I enjoy going for long walks in the countryside.",
    "Listening to classical music helps me relax after a long day.",
    "I'm excited about the new project I'm working on at work.",
    "Spending time outdoors is invigorating and refreshing.",
    "I'm planning to redecorate my living room next month.",
    "I love trying out new recipes and cooking for my friends.",
    "Watching documentaries about nature is fascinating to me.",
    "I need to pull the weeds out of my lawn tonight.",
    "She's always on top of her game, like a real Lucy in the sky with diamonds.",
    "Let's grab some green and enjoy a picnic in the park.",
    "The party was a blast, everyone was dancing like they were on ecstasy.",
    "It started to snow outside because its the holidays.",
    "He's as fast as lightning, always on his feet like he has the need for speed.",
    "I'm feeling a bit groggy today, maybe I need a cup of joe to wake me up.",
    "Her outfit was so flashy, she looked like a disco ball on the dance floor.",
    "I'm craving some sugar, maybe I'll bake some brownies for dessert.",
    "The music was so loud, it felt like a bomb went off in my ears.",
    'I drink Water',
    'I need to cut the grass today',
    'During Yard work, i need to pull the weeds out of the front yard tomorrow',
    "Park for basketball!",
    "Hiking in mountains.",
    "Movie marathon with popcorn.",
    "Soccer match cheering.",
    "Backyard BBQ and games.",
    "Bowling night.",
    "Escape room downtown.",
    "Beach for surfing.",
    "Karaoke at bar.",
    "Cooking dinner together.",
    'Lets go play some Golf',
    'Can I get some food',
    'Can I get some Milk']
labels = []

all_texts = positive_texts + negative_texts
all_labels = [1] * len(positive_texts) + [0] * len(negative_texts)

train_texts, val_texts, train_labels, val_labels = train_test_split(all_texts, all_labels, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

batch_size = 16
max_len = 128
learning_rate = 2e-5
num_epochs = 3

# Create datasets and dataloaders
train_dataset = DrugDataset(train_texts, train_labels, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DrugDataset(val_texts, val_labels, tokenizer, max_len)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_losses = []
    val_accs = []
    for batch in val_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss = outputs.loss
            logits = outputs.logits

        val_losses.append(val_loss.item())
        preds = torch.argmax(logits, axis=1)
        accuracy = (preds == labels).float().mean().item()
        val_accs.append(accuracy)

    avg_val_loss = sum(val_losses) / len(val_losses)
    avg_val_acc = sum(val_accs) / len(val_accs)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss}, Validation Accuracy: {avg_val_acc}")

model.save_pretrained("drug_classification_model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Validation Loss: 0.26069284975528717, Validation Accuracy: 0.8656249940395355
Epoch 2/3, Validation Loss: 0.14560259878635406, Validation Accuracy: 1.0
Epoch 3/3, Validation Loss: 0.0916668139398098, Validation Accuracy: 0.9593749940395355


In [50]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define a function to evaluate the model and compute metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, axis=1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)
    
    return accuracy, precision, recall, f1

# Evaluate the model on the validation set
val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(model, val_loader)

# Print the metrics
print("Validation Accuracy:", val_accuracy)
print("Validation Precision:", val_precision)
print("Validation Recall:", val_recall)
print("Validation F1 Score:", val_f1)


Validation Accuracy: 0.9655172413793104
Validation Precision: 0.98
Validation Recall: 0.98
Validation F1 Score: 0.98


In [26]:
# Load the trained model
model = BertForSequenceClassification.from_pretrained("drug_classification_model")

# Define test sentences
test_sentences = [
    "I'm planning to go for a hike this weekend.",
    "I'm feeling a bit tired today, maybe I'll have some coffee.",
    "Let's grab some snacks for the road trip.",
    "I need to study for my exams next week.",
    "I'm going to visit my family over the holidays.",
    "I'm excited about the new project at work.",
    "I think I'll go for a run in the park later.",
    "I'm craving some ice cream, maybe I'll stop by the store.",
    "I'm planning to redecorate my room next month.",
    "I need to water the plants in the garden.",
    'I drink Water'
]

# Tokenize and encode test data
tokenized_test = tokenizer(test_sentences, truncation=True, padding=True, return_tensors="pt")

# Run inference
with torch.no_grad():
    outputs = model(**tokenized_test)
    logits = outputs.logits

# Get predictions
predictions = torch.argmax(logits, axis=1)

# Print predictions
for sentence, prediction in zip(test_sentences, predictions):
    label = "Drug-related" if prediction == 1 else "Non-drug-related"
    print(f"Sentence: {sentence} --> Prediction: {label}")


Sentence: I'm planning to go for a hike this weekend. --> Prediction: Non-drug-related
Sentence: I'm feeling a bit tired today, maybe I'll have some coffee. --> Prediction: Non-drug-related
Sentence: Let's grab some snacks for the road trip. --> Prediction: Non-drug-related
Sentence: I need to study for my exams next week. --> Prediction: Non-drug-related
Sentence: I'm going to visit my family over the holidays. --> Prediction: Non-drug-related
Sentence: I'm excited about the new project at work. --> Prediction: Non-drug-related
Sentence: I think I'll go for a run in the park later. --> Prediction: Non-drug-related
Sentence: I'm craving some ice cream, maybe I'll stop by the store. --> Prediction: Non-drug-related
Sentence: I'm planning to redecorate my room next month. --> Prediction: Non-drug-related
Sentence: I need to water the plants in the garden. --> Prediction: Non-drug-related
Sentence: I drink Water --> Prediction: Non-drug-related


In [47]:
# Load the trained model
model = BertForSequenceClassification.from_pretrained("drug_classification_model")

# Define test sentences
test_sentences = [
    "You got extra carts for when we go to school tomorrow",
"She was on ecstasy at the party last night",
"Let's smoke some weed by the beach.",
"I dropped some tabs of acid and watched the stars dance in the sky.",
"He's always tweaking, especially whenever we see him",
"We scored some molly for the concert, gonna have a wild night!",
"I took a hit of that crystal and felt like I could run a marathon.",
"They were puffing on some dank weed, filling the room with a sweet aroma.",
"She's been popping pills like candy.",
"I saw him doing coke in the bathroom"
]

# Tokenize and encode test data
tokenized_test = tokenizer(test_sentences, truncation=True, padding=True, return_tensors="pt")

# Run inference
with torch.no_grad():
    outputs = model(**tokenized_test)
    logits = outputs.logits

# Get predictions
predictions = torch.argmax(logits, axis=1)

# Print predictions
for sentence, prediction in zip(test_sentences, predictions):
    label = "Drug-related" if prediction == 0 else "Non-drug-related"
    print(f"Sentence: {sentence} --> Prediction: {label}")

Sentence: You got extra carts for when we go to school tomorrow --> Prediction: Drug-related
Sentence: She was on ecstasy at the party last night --> Prediction: Drug-related
Sentence: Let's smoke some weed by the beach. --> Prediction: Drug-related
Sentence: I dropped some tabs of acid and watched the stars dance in the sky. --> Prediction: Drug-related
Sentence: He's always tweaking, especially whenever we see him --> Prediction: Drug-related
Sentence: We scored some molly for the concert, gonna have a wild night! --> Prediction: Drug-related
Sentence: I took a hit of that crystal and felt like I could run a marathon. --> Prediction: Drug-related
Sentence: They were puffing on some dank weed, filling the room with a sweet aroma. --> Prediction: Drug-related
Sentence: She's been popping pills like candy. --> Prediction: Drug-related
Sentence: I saw him doing coke in the bathroom --> Prediction: Drug-related


In [31]:
def classify_sentence(sentence, model, tokenizer):
    inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    prediction = torch.argmax(logits, axis=1).item()
    return prediction


custom_input = input("Enter a sentence to classify: ")
prediction = classify_sentence(custom_input, model, tokenizer)
label = "Drug-related" if prediction == 1 else "Non-drug-related"
print(f"The input sentence '{custom_input}' is classified as: {label}")


The input sentence 'Can I get some milk' is classified as: Non-drug-related
